In [ ]:
````xml
<VSCode.Cell language="markdown">
# 🔍 Day 4.5 - Grad-CAM Visualization (Model Interpretability)

## 🎯 Learning Objectives

In this notebook, you'll:
1. **Understand Grad-CAM** - Gradient-weighted Class Activation Mapping
2. **Visualize what the CNN focuses on** when making predictions
3. **Generate heatmaps** overlaid on MRI images
4. **Interpret model decisions** for each tumor type
5. **Enhance explainability** for medical AI applications

---

## 🧠 Theory: Grad-CAM

### What Is Grad-CAM?

**Grad-CAM** = **Grad**ient-weighted **C**lass **A**ctivation **M**apping

It visualizes which regions of an image the CNN focuses on for a specific prediction.

### How It Works:

1. **Forward pass**: Get prediction for an image
2. **Backward pass**: Calculate gradients of target class w.r.t. last conv layer
3. **Weight feature maps**: By average gradients
4. **Create heatmap**: Weighted combination of feature maps
5. **Overlay on image**: Show where model "looks"

### Why It Matters for Medical AI:

| Aspect | Importance |
|--------|------------|
| **Trust** | Doctors need to see *why* the model predicts something |
| **Validation** | Verify model focuses on tumor, not artifacts |
| **Error Analysis** | Understand why misclassifications happen |
| **Regulatory** | FDA/CE marking requires explainability |

---
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🔧 Setup
</VSCode.Cell>
<VSCode.Cell language="python">
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
import seaborn as sns
from datetime import datetime

# Add src to path
sys.path.insert(0, '../..')

# Check TensorFlow
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Set style
sns.set_style('white')
plt.rcParams['figure.figsize'] = (16, 10)

print("\n✅ Libraries imported successfully")
print(f"⏰ Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 📂 Define Paths
</VSCode.Cell>
<VSCode.Cell language="python">
# Model path
MODEL_PATH = '../../outputs/models/model_cnn_best.h5'

# Data paths
TEST_CSV = '../../outputs/data_splits/test_split.csv'

# Output path
VIZ_DIR = '../../outputs/visualizations'
os.makedirs(VIZ_DIR, exist_ok=True)

# Class names
CLASS_NAMES = {0: 'Meningioma', 1: 'Glioma', 2: 'Pituitary'}

print("✅ Paths configured")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 💾 Load Model
</VSCode.Cell>
<VSCode.Cell language="python">
print(f"📥 Loading model from: {MODEL_PATH}\n")

if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully")
    
    # Print layer names to find last conv layer
    print("\n📊 Model Layers:")
    for i, layer in enumerate(model.layers):
        print(f"   {i}: {layer.name} ({type(layer).__name__})")
else:
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🎯 Identify Last Convolutional Layer

For Grad-CAM, we need the last convolutional layer (before Flatten).
</VSCode.Cell>
<VSCode.Cell language="python">
# Find last convolutional layer
last_conv_layer_name = None
for layer in reversed(model.layers):
    if 'conv' in layer.name.lower():
        last_conv_layer_name = layer.name
        break

print(f"🔍 Last convolutional layer: {last_conv_layer_name}")

# Verify it exists
last_conv_layer = model.get_layer(last_conv_layer_name)
print(f"   Output shape: {last_conv_layer.output_shape}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🎨 Grad-CAM Implementation
</VSCode.Cell>
<VSCode.Cell language="python">
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generate Grad-CAM heatmap for a given image.
    
    Args:
        img_array: Input image array (1, H, W, C)
        model: Keras model
        last_conv_layer_name: Name of last conv layer
        pred_index: Class index to visualize (None = predicted class)
        
    Returns:
        heatmap: Grad-CAM heatmap (H, W)
    """
    # Create a model that maps input to activations of last conv layer and output predictions
    grad_model = Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # Compute gradient of predicted class w.r.t. feature maps
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    
    # Gradients of the output neuron w.r.t. output feature map
    grads = tape.gradient(class_channel, conv_outputs)
    
    # Average gradients spatially (global average pooling)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight feature maps by gradients
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize heatmap to [0, 1]
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()


def overlay_heatmap_on_image(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    """
    Overlay Grad-CAM heatmap on original image.
    
    Args:
        img: Original image (H, W) or (H, W, 3)
        heatmap: Grad-CAM heatmap (H, W)
        alpha: Transparency of heatmap
        colormap: OpenCV colormap
        
    Returns:
        superimposed_img: Image with heatmap overlay
    """
    # Resize heatmap to match image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    
    # Convert image to RGB if grayscale
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
    # Ensure image is uint8
    if img.dtype != np.uint8:
        img = np.uint8(255 * img)
    
    # Superimpose heatmap on image
    superimposed_img = cv2.addWeighted(img, 1-alpha, heatmap, alpha, 0)
    
    return superimposed_img


print("✅ Grad-CAM functions defined")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🖼️ Load Sample Images

Select representative images from each class.
</VSCode.Cell>
<VSCode.Cell language="python">
# Load test data
test_df = pd.read_csv(TEST_CSV)

# Select 2 samples per class
samples_per_class = {}
for class_label in [1, 2, 3]:
    class_samples = test_df[test_df['label'] == class_label].sample(n=2, random_state=42)
    samples_per_class[class_label] = class_samples

print("📊 Selected samples:")
for class_label, samples in samples_per_class.items():
    print(f"   Class {class_label} ({CLASS_NAMES[class_label-1]}): {len(samples)} samples")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🔍 Generate Grad-CAM Visualizations
</VSCode.Cell>
<VSCode.Cell language="python">
def preprocess_image(img_path, target_size=(128, 128)):
    """Load and preprocess image for model."""
    img = image.load_img(img_path, color_mode='grayscale', target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize
    return img_array, img


# Generate Grad-CAM for all samples
gradcam_results = []

for class_label, samples in samples_per_class.items():
    for idx, row in samples.iterrows():
        img_path = row['filepath']
        true_label = row['label']
        
        # Load and preprocess
        img_array, original_img = preprocess_image(img_path)
        
        # Get prediction
        predictions = model.predict(img_array, verbose=0)
        pred_class = np.argmax(predictions[0])
        confidence = predictions[0][pred_class] * 100
        
        # Generate Grad-CAM heatmap
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=pred_class)
        
        # Overlay on image
        original_img_array = np.array(original_img)
        superimposed = overlay_heatmap_on_image(original_img_array, heatmap, alpha=0.5)
        
        # Store results
        gradcam_results.append({
            'img_path': img_path,
            'original_img': original_img_array,
            'heatmap': heatmap,
            'superimposed': superimposed,
            'true_label': true_label,
            'pred_label': pred_class + 1,
            'confidence': confidence,
            'true_name': CLASS_NAMES[true_label - 1],
            'pred_name': CLASS_NAMES[pred_class]
        })

print(f"✅ Generated Grad-CAM for {len(gradcam_results)} images")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 📊 Visualize Results

Display original image, heatmap, and overlay for each sample.
</VSCode.Cell>
<VSCode.Cell language="python">
# Create comprehensive visualization
n_samples = len(gradcam_results)
fig, axes = plt.subplots(n_samples, 3, figsize=(14, 4*n_samples))

if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, result in enumerate(gradcam_results):
    # Original image
    axes[i, 0].imshow(result['original_img'], cmap='gray')
    axes[i, 0].axis('off')
    axes[i, 0].set_title(
        f"Original\nTrue: {result['true_name']}",
        fontsize=11, fontweight='bold'
    )
    
    # Heatmap
    axes[i, 1].imshow(result['heatmap'], cmap='jet')
    axes[i, 1].axis('off')
    axes[i, 1].set_title(
        f"Grad-CAM Heatmap\nPred: {result['pred_name']}",
        fontsize=11, fontweight='bold'
    )
    
    # Overlay
    axes[i, 2].imshow(result['superimposed'])
    axes[i, 2].axis('off')
    
    # Color-code title based on correctness
    color = 'green' if result['true_label'] == result['pred_label'] else 'red'
    axes[i, 2].set_title(
        f"Overlay\nConfidence: {result['confidence']:.1f}%",
        fontsize=11, fontweight='bold', color=color
    )

plt.suptitle('Grad-CAM Visualizations: Model Attention Regions', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

# Save
gradcam_all_path = os.path.join(VIZ_DIR, 'day4_05_gradcam_all_samples.png')
plt.savefig(gradcam_all_path, dpi=300, bbox_inches='tight')
print(f"✅ Grad-CAM visualization saved: {gradcam_all_path}")
plt.show()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🎯 Class-Specific Grad-CAM Analysis

Show Grad-CAM for each tumor type separately.
</VSCode.Cell>
<VSCode.Cell language="python">
# Create per-class visualization
for class_label in [1, 2, 3]:
    class_results = [r for r in gradcam_results if r['true_label'] == class_label]
    
    if len(class_results) == 0:
        continue
    
    fig, axes = plt.subplots(len(class_results), 3, figsize=(14, 4*len(class_results)))
    
    if len(class_results) == 1:
        axes = axes.reshape(1, -1)
    
    for i, result in enumerate(class_results):
        # Original
        axes[i, 0].imshow(result['original_img'], cmap='gray')
        axes[i, 0].axis('off')
        axes[i, 0].set_title('Original Image', fontsize=11)
        
        # Heatmap
        axes[i, 1].imshow(result['heatmap'], cmap='jet')
        axes[i, 1].axis('off')
        axes[i, 1].set_title('Attention Heatmap', fontsize=11)
        
        # Overlay
        axes[i, 2].imshow(result['superimposed'])
        axes[i, 2].axis('off')
        
        color = 'green' if result['true_label'] == result['pred_label'] else 'red'
        axes[i, 2].set_title(
            f"Pred: {result['pred_name']} ({result['confidence']:.1f}%)",
            fontsize=11, color=color, fontweight='bold'
        )
    
    class_name = CLASS_NAMES[class_label - 1]
    plt.suptitle(f'Grad-CAM for {class_name} Tumors', 
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    
    # Save
    class_path = os.path.join(VIZ_DIR, f'day4_05_gradcam_class_{class_label}_{class_name.lower()}.png')
    plt.savefig(class_path, dpi=300, bbox_inches='tight')
    print(f"✅ Class {class_label} Grad-CAM saved: {class_path}")
    plt.show()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🔍 Interpretation Guide

### How to Read Grad-CAM Heatmaps:

| Color | Meaning |
|-------|---------|
| **Red/Yellow** | High activation - model focuses here |
| **Blue/Purple** | Low activation - model ignores |
| **Green** | Medium activation |

### What to Look For:

1. **Correct Focus**: Is the heatmap centered on the tumor?
2. **Artifacts**: Does model focus on image borders or noise?
3. **Class Differences**: Do different tumor types activate different regions?

### Medical Validation:

✅ **Good**: Model focuses on tumor region  
✅ **Good**: Attention aligns with radiologist annotations  
❌ **Bad**: Model focuses on image artifacts  
❌ **Bad**: Attention on non-tumor regions  

---
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 📊 Analysis: Where Does the Model Look?
</VSCode.Cell>
<VSCode.Cell language="python">
# Analyze attention patterns
print("🔍 Grad-CAM Analysis:\n")
print("="*60)

for class_label in [1, 2, 3]:
    class_results = [r for r in gradcam_results if r['true_label'] == class_label]
    
    if len(class_results) == 0:
        continue
    
    correct = sum(1 for r in class_results if r['true_label'] == r['pred_label'])
    accuracy = (correct / len(class_results)) * 100
    avg_confidence = np.mean([r['confidence'] for r in class_results])
    
    # Calculate average heatmap intensity in center vs edges
    center_intensities = []
    edge_intensities = []
    
    for r in class_results:
        heatmap = r['heatmap']
        h, w = heatmap.shape
        
        # Center region (middle 50%)
        center_h_start, center_h_end = h//4, 3*h//4
        center_w_start, center_w_end = w//4, 3*w//4
        center = heatmap[center_h_start:center_h_end, center_w_start:center_w_end]
        center_intensities.append(np.mean(center))
        
        # Edge region (outer 25%)
        edge_thickness = min(h, w) // 8
        edge = np.concatenate([
            heatmap[:edge_thickness, :].flatten(),
            heatmap[-edge_thickness:, :].flatten(),
            heatmap[:, :edge_thickness].flatten(),
            heatmap[:, -edge_thickness:].flatten()
        ])
        edge_intensities.append(np.mean(edge))
    
    avg_center = np.mean(center_intensities)
    avg_edge = np.mean(edge_intensities)
    center_edge_ratio = avg_center / avg_edge if avg_edge > 0 else 0
    
    class_name = CLASS_NAMES[class_label - 1]
    print(f"\n{class_name}:")
    print(f"   Samples analyzed: {len(class_results)}")
    print(f"   Prediction accuracy: {accuracy:.1f}%")
    print(f"   Average confidence: {avg_confidence:.1f}%")
    print(f"   Avg center activation: {avg_center:.3f}")
    print(f"   Avg edge activation: {avg_edge:.3f}")
    print(f"   Center/Edge ratio: {center_edge_ratio:.2f}x")
    
    if center_edge_ratio > 2.0:
        print(f"   ✅ Model focuses on center (likely tumor region)")
    elif center_edge_ratio > 1.2:
        print(f"   ⚠️ Model moderately focused on center")
    else:
        print(f"   ❌ Model may focus on edges (check for artifacts)")

print("\n" + "="*60)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 💡 Interpretation & Insights
</VSCode.Cell>
<VSCode.Cell language="python">
print("\n💡 KEY INSIGHTS FROM GRAD-CAM ANALYSIS:\n")
print("="*60)

# Calculate overall statistics
total_correct = sum(1 for r in gradcam_results if r['true_label'] == r['pred_label'])
total_samples = len(gradcam_results)
overall_accuracy = (total_correct / total_samples) * 100

all_center_edge_ratios = []
for r in gradcam_results:
    heatmap = r['heatmap']
    h, w = heatmap.shape
    center = heatmap[h//4:3*h//4, w//4:3*w//4]
    edge_thickness = min(h, w) // 8
    edge = np.concatenate([
        heatmap[:edge_thickness, :].flatten(),
        heatmap[-edge_thickness:, :].flatten(),
        heatmap[:, :edge_thickness].flatten(),
        heatmap[:, -edge_thickness:].flatten()
    ])
    ratio = np.mean(center) / np.mean(edge) if np.mean(edge) > 0 else 0
    all_center_edge_ratios.append(ratio)

avg_ratio = np.mean(all_center_edge_ratios)

print(f"\n1️⃣ Model Attention Pattern:")
print(f"   Average center/edge activation ratio: {avg_ratio:.2f}x")
if avg_ratio > 2.0:
    print(f"   ✅ Model correctly focuses on central tumor regions")
elif avg_ratio > 1.2:
    print(f"   ⚠️ Model shows moderate central focus")
else:
    print(f"   ❌ Model may be distracted by edge artifacts")

print(f"\n2️⃣ Prediction Quality:")
print(f"   Correct predictions: {total_correct}/{total_samples} ({overall_accuracy:.1f}%)")

correct_confidences = [r['confidence'] for r in gradcam_results if r['true_label'] == r['pred_label']]
incorrect_confidences = [r['confidence'] for r in gradcam_results if r['true_label'] != r['pred_label']]

if correct_confidences:
    print(f"   Avg confidence (correct): {np.mean(correct_confidences):.1f}%")
if incorrect_confidences:
    print(f"   Avg confidence (incorrect): {np.mean(incorrect_confidences):.1f}%")

print(f"\n3️⃣ Clinical Implications:")
print(f"   ✅ Grad-CAM provides visual explanation of predictions")
print(f"   ✅ Helps radiologists verify model decisions")
print(f"   ✅ Can identify when model focuses on wrong regions")
print(f"   ✅ Increases trust in AI-assisted diagnosis")

print("\n" + "="*60)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## ✅ Grad-CAM Analysis Complete - Summary

### What We Accomplished:

1. ✅ **Implemented Grad-CAM** from scratch using TensorFlow
2. ✅ **Generated heatmaps** for 6 sample images (2 per class)
3. ✅ **Visualized attention patterns** with overlay images
4. ✅ **Analyzed focus regions** (center vs edges)
5. ✅ **Saved visualizations** for each tumor type

### Key Findings:

**🎯 Model Attention:**
- Average center/edge ratio: {avg_ratio:.2f}x
- Model {'focuses well on tumor regions' if avg_ratio > 2.0 else 'needs attention pattern improvement'}

**💡 Clinical Value:**
- Grad-CAM provides **visual explanation** of CNN decisions
- Helps **validate** that model focuses on relevant anatomy
- Enables **trust** in AI-assisted diagnosis
- **Regulatory requirement** for medical AI deployment

### Interpretation:

- **Red/Yellow regions**: Model pays most attention here
- **Should align with**: Tumor location in MRI
- **If misaligned**: May indicate spurious correlations or artifacts

### Medical AI Best Practices:

✅ **Always use Grad-CAM** for medical imaging models  
✅ **Validate with radiologists** that attention makes sense  
✅ **Check for artifacts** (image borders, text, etc.)  
✅ **Document findings** in model cards  

---

**Date:** October 22, 2025  
**Status:** ✅ Completed  
**Samples Analyzed:** {total_samples}  
**Avg Center/Edge Ratio:** {avg_ratio:.2f}x  

---

## 📚 References

1. Selvaraju et al., "Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization" (2017)
2. FDA Guidance on AI/ML in Medical Devices
3. European AI Act - Medical AI Requirements
</VSCode.Cell>
````